# Ficha 5

### Enunciado

### Questões

1. Elabore um pequeno caderno de requisitos que possa justificar o esquema lógico apresentado.

É pedido um sistema de base de dados que permita registar os médicos em atividade na clínica e os seus respetivos dados (nome, data de início de serviço, data de nascimento, morada e especialidade). Cada médico deve pertencer a uma especialidade.

Deverá também ser possível criar entradas para as diferentes especialidades que podem ser consultadas na clinica, nomeadamente a sua designação e preço por consulta.

O sistema também deverá permitir registar os pacientes que são consultados na clínica e respetivos dados (nome, data de nascimento e morada)

É também pedido um sistema de registo de consultas, onde seja possível registar o paciente que foi consultado, por quem foi consultado e quando foi consultado e quanto pagou pela consulta.

2. Desenhe um possível esquema conceptual equivalente ao esquema lógico apresentado na Figura 1.

![](misc/ficha-5-chen-diag.png)

3. tilizando álgebra relacional, desenvolva as expressões necessárias, e respetivas árvores de resolução, que possam responder às seguintes questões:

* a) Qual o nome dos médicos com mais de 10 anos de serviço?

    $ \pi $<sub>nome</sub>($\sigma$<sub>data_inicio_servico < 2014</sub>(medico))

* b) Qual é o nome de todos os médicos e a respetiva especialidade que cada um exerce?

    $ \pi $ <sub>nome,especialidade</sub>($medico \Join especialidade$)

* c) Qual é o nome e a morada dos pacientes residentes na localidade de Braga?
    
    $\pi$<sub>nome,morada</sub>($paciente \ltimes \sigma$<sub>localidade=Braga</sub>(codigo postal))

* d) Qual é o nome dos médicos da especialidade de Oftalmologia?

    $\pi$<sub>nome</sub>(medico $\ltimes \sigma$ <sub>designacao='Oftalmologia'</sub>(especialidade))

* e) Qual é o nome e a idade dos médicos com mais de 40 anos de idade da especialidade de Clínica Geral?

    $\pi$<sub>nome,especialidade,2024 - data nascimento</sub>((médico $\Join \sigma$ <sub>designacao='Clinica Geral'</sub>(especialidade)) $\ltimes \sigma$<sub>data nascimento < 1984</sub>(medico)   )

* f) Qual é o nome dos médicos da especilidade de Oftalmologia que consultaram pacientes da localidade de Braga?

    $\pi$<sub>nome,especialidade</sub> ( 
        (medico $\ltimes \sigma$ <sub>designacao='Oftalmologia'</sub>(especialidade))
        $\ltimes$
        (consulta $\ltimes$ 
        (paciente $\ltimes \sigma$<sub>localidade=Braga</sub>(codigo postal)))
    )

* g) Qual é o nome e os anos de serviço dos médicos com mais de 50 anos de idade que deram consultas a partir das 12h a pacientes com menos de 20 anos de idade? 
    
    $\pi$<sub>nome,2024 - data inicio servico</sub>(
        $\sigma$<sub>data_nascimento < 1974</sub>(medico)
        $\ltimes$
        ($\sigma$<sub>data_hora > 12:00</sub>(consulta)
        $\ltimes$
        ($\sigma$ <sub>data_nascimento < 2004</sub>(paciente) $\ltimes$ $\sigma$<sub>localidade=Braga</sub>(codigo postal))))

* h) Qual é o nome e a idade dos pacientes com mais de 10 anos de idade que nunca foram consultados na especialidade de Oftalmologia?

    $\pi$<sub>nome,2024 - data_nascimento</sub>(
        $\sigma$ <sub>data_nascimento < 2014</sub>(paciente) - $\sigma$<sub>designacao = 'Oftalmologia'</sub>(especialidade))

4. Seguem abaixo os scripts gerados pelo `Forward Engineering` ao passar o esquema relacional para a base de dados:

``` sql

-- MySQL Workbench Forward Engineering

SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;
SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0;
SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';

-- -----------------------------------------------------
-- Schema Clinica
-- -----------------------------------------------------

-- -----------------------------------------------------
-- Schema Clinica
-- -----------------------------------------------------
CREATE SCHEMA IF NOT EXISTS `Clinica` DEFAULT CHARACTER SET utf8 ;
USE `Clinica` ;

-- -----------------------------------------------------
-- Table `Clinica`.`ESPECIALIDADE`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `Clinica`.`ESPECIALIDADE` (
  `id_especialidade` INT(3) NOT NULL,
  `designacao` VARCHAR(250) NOT NULL,
  `preco` DECIMAL(6,2) NOT NULL,
  PRIMARY KEY (`id_especialidade`))
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Table `Clinica`.`CODIGO_POSTAL`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `Clinica`.`CODIGO_POSTAL` (
  `codigo_postal` VARCHAR(8) NOT NULL,
  `localidade` VARCHAR(250) NOT NULL,
  PRIMARY KEY (`codigo_postal`))
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Table `Clinica`.`MEDICO`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `Clinica`.`MEDICO` (
  `id_medico` INT(6) NOT NULL,
  `nome` VARCHAR(250) NOT NULL,
  `morada` VARCHAR(250) NOT NULL,
  `codigo_postal` VARCHAR(8) NOT NULL,
  `data_nascimento` DATE NOT NULL,
  `especialidade` INT(3) NOT NULL,
  `data_inicio_servico` DATE NOT NULL,
  INDEX `fk_med_idx` (`especialidade` ASC),
  INDEX `fk_cp_idx` (`codigo_postal` ASC),
  PRIMARY KEY (`id_medico`),
  CONSTRAINT `fk_especialidade1`
    FOREIGN KEY (`especialidade`)
    REFERENCES `Clinica`.`ESPECIALIDADE` (`id_especialidade`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_codigo_postal1`
    FOREIGN KEY (`codigo_postal`)
    REFERENCES `Clinica`.`CODIGO_POSTAL` (`codigo_postal`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Table `Clinica`.`PACIENTE`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `Clinica`.`PACIENTE` (
  `id_paciente` INT(6) NOT NULL,
  `nome` VARCHAR(250) NOT NULL,
  `morada` VARCHAR(250) NOT NULL,
  `codigo_postal` VARCHAR(8) NOT NULL,
  `data_nascimento` DATE NOT NULL,
  INDEX `fk_paci_idx` (`codigo_postal` ASC),
  PRIMARY KEY (`id_paciente`),
  CONSTRAINT `fk_codigo_postal2`
    FOREIGN KEY (`codigo_postal`)
    REFERENCES `Clinica`.`CODIGO_POSTAL` (`codigo_postal`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;


-- -----------------------------------------------------
-- Table `Clinica`.`CONSULTA`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `Clinica`.`CONSULTA` (
  `id_medico` INT(6) NOT NULL,
  `id_paciente` INT(6) NOT NULL,
  `data_hora` DATETIME NOT NULL,
  `preco` DECIMAL(8,2) NOT NULL,
  PRIMARY KEY (`id_medico`, `id_paciente`, `data_hora`),
  INDEX `FK_cons02_idx` (`id_paciente` ASC),
  CONSTRAINT `fk_medico1`
    FOREIGN KEY (`id_medico`)
    REFERENCES `Clinica`.`MEDICO` (`id_medico`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_paciente1`
    FOREIGN KEY (`id_paciente`)
    REFERENCES `Clinica`.`PACIENTE` (`id_paciente`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;


SET SQL_MODE=@OLD_SQL_MODE;
SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS;
SET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS;


```

